In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer


In [35]:
true=pd.read_csv("True.csv")
fake=pd.read_csv("Fake.csv")

In [36]:
true['label']='True'
fake['label']='Fake'

In [37]:
df=pd.concat([true,fake])

In [38]:
#cleaning the string
def clean_str(string):
    string = re.sub(r'http\S+', 'link', string) # replace links by generic text link
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)

    cleanr = re.compile('<.*?>')

    string = re.sub(r'\d+', '', string)
    string = re.sub(cleanr, '', string)
    string = re.sub("'", '', string)
    string = re.sub(r'\W+', ' ', string)
    string = string.replace('_', '')

    return string.strip().lower()

In [39]:
stop_words=set(stopwords.words('english'))

def remove_stopwords(word_list):
    no_stop_words=[w for w in word_list if not w in stop_words]
    return no_stop_words

In [40]:
sentences=[]
text=df.text
labels=[]
for row in text:
    #print(row)
    row=clean_str(row)
    #print(row)
    wo_stop=remove_stopwords(row.split(" "))
    wo_stop=" ".join(wo_stop)
    #print(wo_stop+"\n\n\n")
    sentences.append(wo_stop)
for label in df.label:
    labels.append(label)
#print(sentences)
print(len(sentences))
print(len(labels))

44898
44898


In [41]:
#hyperparameters
train_ratio=0.7
vocab_size=10000
embedding_dim=32
max_length=32
trunc_type='post'
oov_tok="<OOV>"
batch_size=128


In [42]:
from sklearn.model_selection import train_test_split

X = df['title']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [43]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_tokenizer=Tokenizer(num_words=vocab_size, oov_token=oov_tok)
train_tokenizer.fit_on_texts(X_train)
train_sequences=train_tokenizer.texts_to_sequences(X_train)
train_padded=pad_sequences(train_sequences, maxlen=max_length, truncating=trunc_type)

val_sequences=train_tokenizer.texts_to_sequences(X_test)
val_padded=pad_sequences(val_sequences, maxlen=max_length, truncating=trunc_type)

In [44]:
X_train=np.array(train_padded)

X_test=np.array(val_padded)


In [45]:
from sklearn.linear_model import LogisticRegression

In [46]:
model=LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

In [47]:
pred=model.predict(X_test)

In [48]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,pred))

[[4489 2507]
 [ 621 5853]]


In [49]:
print(metrics.classification_report(y_test,pred))

              precision    recall  f1-score   support

        Fake       0.88      0.64      0.74      6996
        True       0.70      0.90      0.79      6474

    accuracy                           0.77     13470
   macro avg       0.79      0.77      0.77     13470
weighted avg       0.79      0.77      0.76     13470

